# Exercices Seance 06

## 1/ XML to CSV

Écrire une fonction qui lit les 5 fichiers XML de Balzac, créé et sauvegarde un dataframe avec 8 colonnes : titre, auteur, gender, genre, canon, date_publication, date_naissance, date_mort


In [10]:
import pandas as pd
from lxml import etree
from glob import glob
from tqdm.notebook import tqdm

In [7]:
path_to_folder_xml = "/home/crazyjeannot/Documents/cours/2024/Python4DH/data/xml/*.xml"

In [3]:
def info_xml(fichier_xml):
    arbre = etree.parse(fichier_xml)
    dico_res = {'titre':None, 'auteur':None, 'gender':None, 'genre':None, 'canon':None, 'date_publication':None, 'date_naissance':None, 'date_mort':None}
    dico_res["titre"] = arbre.find(".//title").text
    dico_res["auteur"] = arbre.find(".//author").get('name')
    dico_res["gender"] = arbre.find(".//author").get('sex')
    dico_res["date_naissance"] = arbre.find(".//author").get('from')
    dico_res["date_mort"] = arbre.find(".//author").get('to')
    dico_res["date_publication"] = arbre.findall(".//date")[1].get('when')
    dico_res["canon"] = arbre.find('.//profileDesc').get('tag')
    dico_res["genre"] = arbre.find('.//term').text
    return dico_res


In [4]:
def trouve_info(path_to_xml):
    tree = etree.parse(path_to_xml)

    titre = tree.find(".//title").text
    auteur = tree.find(".//author").get('name')
    gender = tree.find(".//author").get('sex')
    date_naissance = tree.find(".//author").get('from')
    date_mort = tree.find(".//author").get('to')
    date_publication = tree.findall(".//date")[1].get('when')
    canon = tree.find('.//profileDesc').get('tag')
    genre = ' '.join([element.text for element in tree.findall('.//term')]) 

    return titre, auteur, gender, genre, canon, date_publication, date_naissance, date_mort


In [8]:
def moulinette(path_to_folder_xml):
    df_corpus = pd.DataFrame([], columns=['titre', "auteur", "gender", "genre", "canon", "date_publication", "date_naissance", "date_mort"])
    for doc in tqdm(glob(path_to_folder_xml)):
        titre, auteur, gender, genre, canon, date_publication, date_naissance, date_mort = trouve_info(doc) 
    
        df_roman = pd.DataFrame([(titre, auteur, gender, genre, canon, date_publication, date_naissance, date_mort)], columns=['titre', "auteur", "gender", "genre", "canon", "date_publication", "date_naissance", "date_mort"])
        df_corpus = pd.concat([df_corpus, df_roman])
    return df_corpus

In [11]:
df = moulinette(path_to_folder_xml)

  0%|          | 0/5 [00:00<?, ?it/s]

In [12]:
df

,titre,auteur,gender,genre,canon,date_publication,date_naissance,date_mort
0,Le Chef-d'œuvre inconnu,"Balzac, Honoré de",male,cycles et séries nouvelles,canon,1845,1799,1850
0,Eugénie Grandet,"Balzac, Honoré de",male,cycles et séries,canon,1843,1799,1850
0,Le Père Goriot,"Balzac, Honoré de",male,cycles et séries,canon,1843,1799,1850
0,Sarrasine,"Balzac, Honoré de",male,cycles et séries nouvelles,canon,1844,1799,1850
0,Illusions perdues,"Balzac, Honoré de",male,cycles et séries,canon,1843,1799,1850


In [9]:
get_dataframe

<function __main__.get_dataframe(dossier_xml)>

## 2/ XML to txt 

Écrire une fonction qui convertie les 5 fichiers XML en fichiers txt.

In [14]:
import os

In [17]:
path_to_folder_txt = "/home/crazyjeannot/Documents/cours/2024/Python4DH/data/txt/"

In [18]:
def xml_to_txt(path_xml, path_name_res):
    
    chaine = ""
    doc_name = os.path.splitext(os.path.basename(path_xml))[0]
        
    tree = etree.parse(path_xml)

    if tree.findall(".//p"):
        for line in tree.findall(".//p"):
            if line.text:
                chaine += line.text
    else:
        print(doc_name)
                    
    with open(path_name_res+doc_name+".txt", "w", encoding="utf8") as file_out:
        file_out.write(chaine)

In [22]:
def moulinette_xmltotxt(path_name, path_name_res):
    for doc in tqdm(glob(path_name)):
        xml_to_txt(doc, path_name_res)

In [23]:
moulinette_xmltotxt(path_to_folder_xml, path_to_folder_txt)

  0%|          | 0/5 [00:00<?, ?it/s]

## 3/ Exploiter des JSON

À partir des .book (json customisés) envoyés avec ce notebook, écrire une fonction qui ouvre les documents, une autre qui récupère la liste des verbes ('agent', 'patient') et des adjectifs ('mod') associés à ces personnages (les 10 premiers), une autre qui calcule leur fréquence d'apparition relative au nombre total de verbes / adjectifs et retourne le résultat sour la forme d'un dataframe (colonnes = mots, ligne = txt_j_personnage_i, valeurs = fréquence relative).

Enfin définir une fonction main qui appelle ces différentes fonctions, fusionne les dataframes au fur et à mesure puis retourne le résultat sous la forme d'un dataframe (colonnes = mots, ligne = txt_j_personnage_i, valeurs = fréquence relative). 

Sauver ce dataframe final. Me l'envoyer avec votre notebook. 

Vous pouvez utiliser la fonction get_characterization vue dans le cours précédent.

Vous pouvez également utiliser les fonctions déjà définies dans les différents exercices corrigés des semaines précédentes

In [46]:
import json

In [47]:
def get_characterization(booknlp_data):
    list_verb_suj, list_verb_obj, list_adj = [], [], []
    for i in range(10):
        list_verb_suj.append([item['w'] for item in booknlp_data["characters"][i]['agent']])
        list_verb_obj.append([item['w'] for item in booknlp_data["characters"][i]['patient']])
        list_adj.append([item['w'] for item in booknlp_data["characters"][i]['mod']])
    return list_verb_suj, list_verb_obj, list_adj

In [61]:
def load_json(path_files):
    list_res = []
    for file in glob(path_files):
        fichier_courant = open(file, mode='r', encoding='utf-8')
        list_res.append(json.load(fichier_courant))
    return list_res

In [49]:
path_to_folder_json = "/home/crazyjeannot/Documents/cours/2024/Python4DH/data/json/*.book"

In [58]:
import numpy as np

In [92]:
def ratio(book_fichier):
    agent, patient, mod = get_characterization(book_fichier)
    dico = {}
    for prenom in range(10):
        if len(book_fichier['characters'][prenom]['mentions']['proper'])==0:
            nom = prenom
        else:
            nom = book_fichier['characters'][prenom]['mentions']['proper'][0]["n"]

        dico[nom]={"agent":0, "patient":0, "mod":0}
        total_agent = np.sum(len(agent_perso_i) for agent_perso_i in agent)
        total_patient = np.sum(len(patient_perso_i) for patient_perso_i in patient)
        total_mod = np.sum(len(mod_perso_i) for mod_perso_i in mod)

        if total_agent==0:
            total_agent = 1
        if total_patient==0:
            total_patient = 1
        if total_mod==0:
            total_mod = 1
        
        dico[nom]['patient'] = len(patient[prenom])/total_patient
        dico[nom]['mod'] = len(mod[prenom])/total_mod
        dico[nom]['agent'] = len(agent[prenom])/total_agent
    return pd.DataFrame(dico)

In [93]:
def main(path_books):
    valeur_finale = []
    for file in load_json(path_books):
        valeur_finale.append(ratio(file))
    return valeur_finale

In [95]:
main(path_to_folder_json)

/tmp/ipykernel_104668/3155176275.py:11: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  total_agent = np.sum(len(agent_perso_i) for agent_perso_i in agent)
/tmp/ipykernel_104668/3155176275.py:12: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  total_patient = np.sum(len(patient_perso_i) for patient_perso_i in patient)
/tmp/ipykernel_104668/3155176275.py:13: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  total_mod = np.sum(len(mod_perso_i) for mod_perso_i in mod)
/tmp/ipykernel_104668/3155176275.py:11: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a di

[                0         1         2         3         4         5         6  \
 agent    0.246528  0.100694  0.085069  0.149306  0.126736  0.093750  0.055556   
 patient  0.100000  0.088889  0.400000  0.133333  0.044444  0.077778  0.033333   
 mod      0.272727  0.090909  0.272727  0.000000  0.045455  0.045455  0.045455   
 
            « Sido     Mme B   Colette  
 agent    0.062500  0.039931  0.039931  
 patient  0.044444  0.033333  0.044444  
 mod      0.136364  0.045455  0.045455  ,
          Arsène Lupin         1  Ganimard         3         4    Daspry  \
 agent        0.279343  0.118936  0.124413  0.067293  0.097027  0.083725   
 patient      0.283276  0.337884  0.068259  0.071672  0.058020  0.037543   
 mod          0.237288  0.084746  0.135593  0.084746  0.050847  0.101695   
 
          miss Nelly   Danègre         8         9  
 agent      0.072770  0.058685  0.049296  0.048513  
 patient    0.061433  0.044369  0.020478  0.017065  
 mod        0.050847  0.118644  0.050847